### Counterfactual plots

In [ ]:
### Load libraries
library(here) # setting paths
library(rstan) # for reading the modle
library(ggplot2) # for plots
library(matrixStats) # for geting quantiles
library(gridExtra)

In [2]:
# Input paths
path2stats<-here("data","Mean_and_SD.csv") # Environmental variables summary
path2envtab<-here("outputs","Predictor_variables.csv")
path2cfiles<-here("outputs","predictions","counter_factual") # Output files - to list fiels for counterfactual plots

# Output paths

path2cfp<-here("manuscript","Chapter6","Figs","Raster")

In [3]:
crops<-c("rice","millet","buckwheat","wheat","barley")

In [4]:
env_tab<-read.csv(path2envtab,row.names=1)
stats<-read.csv(path2stats,row.names=1)
cffiles <- list.files(path=path2cfiles,full.names=TRUE)

In [ ]:
for (crop in crops){
    print(crop)
    # Subset data for specific crops
    cfiles<-subset(cffiles,lapply(cffiles, grepl,pattern=paste("/",crop,sep=""))==TRUE)
    
    # Load data for counterfactual plots:
    counterfactual_data<-c()
    for(i in 1:length(cfiles)){
    cpredictions<-read.csv(cfiles[i])
        y<-as.numeric(colMeans(cpredictions)) # Get mean predictions for each value of the predictor
        yquant<-data.frame(colQuantiles(as.matrix(cpredictions),probs = c(0.05,0.25,0.75,0.95))) # Get quantile for each value of the predictor
        yquant$mean<-y # combine data
        yquant$value<-seq(from = stats[i,'low_range'], to = stats[i,'high_range'],length.out=1000)
        if(rownames(stats)[i]=="BIO4 Temperature Seasonality"){
          yquant$value<-yquant$value/100
        }
        yquant$name<-rownames(stats)[i]
      
    counterfactual_data[[i]]<-yquant
    }
    
    ### Define labels and range for the plot
    names<-env_tab[rownames(stats),] # panel names
    units <- c("°C","mm","standard deviation of average monthly temperatures\nin °C*100","°C"," mass of carbon per unit area per year") # units
    yrange<-c(min(unlist(lapply(counterfactual_data,function(x) min(x$X5)))),max(unlist(lapply(counterfactual_data,function(x) max(x$X95))))) # predictions range
                                                                                            
     ### Prepare plots for all variables
    p1 <- ggplot(counterfactual_data[[1]], aes(value))+
    geom_ribbon(aes(ymin=exp(X5.), ymax=exp(X95.)),fill="#cce5ff" )+
    geom_ribbon(aes(ymin=exp(X25.), ymax=exp(X75.)),fill="#99cbff" )+
    geom_line(aes(y=exp(mean)),col="#004c99")+
    scale_y_continuous("",expand=c(0,0),limits=exp(yrange),trans='log10')+
    scale_x_continuous(units[1],expand=c(0,0))+
    ggtitle(names[1])+
    theme_classic()+
    theme(title=element_text(size=6))

    p2 <- ggplot(counterfactual_data[[2]], aes(value))+
    geom_ribbon(aes(ymin=exp(X5.), ymax=exp(X95.)),fill="#cce5ff" )+
    geom_ribbon(aes(ymin=exp(X25.), ymax=exp(X75.)),fill="#99cbff" )+
    geom_line(aes(y=exp(mean)),col="#004c99")+
    scale_y_continuous("",expand=c(0,0),limits=exp(yrange),trans='log10')+
    scale_x_continuous(units[2],expand=c(0,0))+
    ggtitle(names[2])+
    theme_classic()+
    theme(title=element_text(size=6))

    p3 <- ggplot(counterfactual_data[[3]], aes(value))+
    geom_ribbon(aes(ymin=exp(X5.), ymax=exp(X95.)),fill="#cce5ff" )+
    geom_ribbon(aes(ymin=exp(X25.), ymax=exp(X75.)),fill="#99cbff" )+
    geom_line(aes(y=exp(mean)),col="#004c99")+
    scale_y_continuous("Predicted suitability (area under cultivation)",expand=c(0,0),limits=exp(yrange),trans='log10')+
    scale_x_continuous(units[3],expand=c(0,0))+
    ggtitle(names[3])+
    theme_classic()+
    theme(axis.title=element_text(size=7),
     title=element_text(size=6))

    p4 <- ggplot(counterfactual_data[[4]], aes(value))+
    geom_ribbon(aes(ymin=exp(X5.), ymax=exp(X95.)),fill="#cce5ff" )+
    geom_ribbon(aes(ymin=exp(X25.), ymax=exp(X75.)),fill="#99cbff" )+
    geom_line(aes(y=exp(mean)),col="#004c99")+
    scale_y_continuous("",expand=c(0,0),limits=exp(yrange),trans='log10')+
    scale_x_continuous(units[4],expand=c(0,0))+
    ggtitle(names[4])+
    theme_classic()+
    theme(title=element_text(size=6))

    p5 <- ggplot(counterfactual_data[[5]], aes(value))+
    geom_ribbon(aes(ymin=exp(X5.), ymax=exp(X95.)),fill="#cce5ff" )+
    geom_ribbon(aes(ymin=exp(X25.), ymax=exp(X75.)),fill="#99cbff" )+
    geom_line(aes(y=exp(mean)),col="#004c99")+
    scale_y_continuous("",expand=c(0,0),limits=exp(yrange),trans='log10')+
    scale_x_continuous(units[5],expand=c(0,0))+
    ggtitle(names[5])+
    theme_classic()+
    theme(title=element_text(size=6))

    # Prepare legend
    p6<- ggplot(counterfactual_data[[1]], aes(value))+
    geom_ribbon(aes(ymin=exp(X5.), ymax=exp(X95.),fill="95% confidence interval"))+
    geom_ribbon(aes(ymin=exp(X25.), ymax=exp(X75.),fill="50% confidence interval" ))+
    scale_fill_manual(values=c("#cce5ff","#99cbff"),name="")+
    geom_line(aes(y=exp(mean),col="Mean predictions"))+
    scale_colour_manual(values=c("#004c99"),name="")+
    theme_classic()

    legend <- cowplot::get_legend(p6)                                                                                       
    
    # Arrange grid and optionally plot in jupyter
    options(repr.plot.width=6,repr.plot.height=6,repr.plot.res=300)
    cfp<-grid.arrange(p1, p2, p3,p4,p5,legend, ncol=2)
    
    ### Save plot
   # height=16 #20.4
   # width=14.3 #17.4
   # ggsave(paste(path2cfp,"/",crop,"_cfp.png",sep=""),cfp,device="png",width = width, height = height,unit="cm",dpi=600)
                                                                                                
}

In [33]:
### Save plot
height=16 #20.4
width=14.3 #17.4
ggsave(paste(path2cfp,"/",crop,"_cfp.png",sep=""),cfp,device="png",width = width, height = height,unit="cm",dpi=600)